In [ ]:
import torch, torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import datasets, transforms
from torchsummary import summary

In [2]:
# HYPERPARAMETERS
T = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))
])
TRAIN_TEST_SPLIT = 0.9
BATCH_SIZE = 64
EPOCHS = 50

Part 1. Loading the Dataset

In [ ]:
dataset = datasets.CIFAR10(root="./CIFAR10", download=True, transform=T)
trainDataset, testDataset = random_split(dataset, [TRAIN_TEST_SPLIT, 1 - TRAIN_TEST_SPLIT])
trainLoader = DataLoader(trainDataset,
                         batch_size=BATCH_SIZE,
                         shuffle=True)
testLoader  = DataLoader(testDataset,
                         batch_size=BATCH_SIZE,
                         shuffle=True)

Dataset CIFAR10
    Number of datapoints: 50000
    Root location: ./CIFAR10
    Split: Train
    StandardTransform
Transform: Compose(
               Resize(size=(32, 32), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=(0.5,), std=(0.5,))
           )


Part 2. Defining the Neural Network Architecture

In [ ]:
class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        # size_out = (size_in + 2 * padding - kernel) / stride
        self.encoder = nn.Sequential( 
            # INPUT 1x32x32
            nn.Conv2d(1, 1, kernel_size=3, stride=2, padding=1),    # 3x16x16
            nn.ReLU(),
            nn.Conv2d(1, 1, kernel_size=3, stride=2, padding=1),    # 6x8x8
        )
        # size_out = (size_in - 1) * stride - 2 * padding + kernel
        self.decoder = nn.Sequential(
            # INPUT 6x8x8
            nn.ConvTranspose2d(1, 1, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(1, 1, kernel_size=4, stride=2, padding=1)
        )
    def forward(self, x):
        z = self.encoder(x)
        return self.decoder(z)

model = AutoEncoder()
model.to("cuda")
summary(model, (1,32,32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 1, 16, 16]              10
              ReLU-2            [-1, 1, 16, 16]               0
            Conv2d-3              [-1, 1, 8, 8]              10
              ReLU-4              [-1, 1, 8, 8]               0
   ConvTranspose2d-5            [-1, 1, 16, 16]              17
              ReLU-6            [-1, 1, 16, 16]               0
   ConvTranspose2d-7            [-1, 1, 32, 32]              17
Total params: 54
Trainable params: 54
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.02
Params size (MB): 0.00
Estimated Total Size (MB): 0.02
----------------------------------------------------------------


Part 3. Train the Model

Part 6. Unsupervised Classification